In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
from datetime import datetime
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"//data//"+p, data_list)) # insert your file
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240818_1925.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240818_1325.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240818_1125.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240815_1759.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240810_1240.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240810_0840.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240627_1720.raw', 'C://Users//jony//Programming//Python//Anubis//anubis////data//datafiles.root', 'C://Users//jony//Programming//Python//Anubis//anubis////data//dataAtlas.root']


In [4]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 1000 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
while processedEvents < max_process_event_chunk:
    processedEvents += 1
    event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    if event_chunk:
        TAnalyser.update_event(event_chunk, processedEvents)
        TAnalyser.readTDCTimeDiffs()
        
outDict = {'totDiffs':TAnalyser.totDiffs,
                    'nDiffs':TAnalyser.nDiffs,
                    'diffHists':TAnalyser.diffHists} 
residEta, residPhi = TAnalyser.Calculate_Residual_and_plot_TDC_Time_Diffs( 
                                                     pdf_filename='output/TDC_time_diffs.pdf', 
                                                     max_itr = 5)

KeyboardInterrupt: 

In [3]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(RTools)
import pandas as pd
interval = 100 # Set your monitoring chunck size
file_path  = ["C:\\Users\\jony\\Programming\\Python\\Anubis\\anubis\\data\\proAnubis_240815_1759.raw", "C:\\Users\\jony\\Programming\\Python\\Anubis\\anubis\\data\\proAnubis_240818_1125.raw"]
fReader = rawFileReader.fileReader(file_path[1]) # load in the classs object
print(file_path[1])
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 1_0000000000 # End the loop early
processedEvents = 0 # Initialisation

hv_file = 'hvScan.csv'  # Replace with your file path
hv_data = pd.read_csv(hv_file)

date = "2024-08-18"
measurement = 0

initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
event_time = max([initial_event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if initial_event_chunk[0].tdcEvents[tdc].time])
print(event_time)
i = 0
while event_time < datetime.strptime("2024-08-18 11:35:54", '%Y-%m-%d %H:%M:%S'):
    fReader.skip_events(2_000)
    i += 1
    initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    if initial_event_chunk:
        event_time = max([initial_event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if initial_event_chunk[0].tdcEvents[tdc].time])
        if i % 1000 == 0:
            print(event_time)
print(event_time)    

reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents)

# Load the CSV file

# Display the first few rows of the dataframe
historgrams = [[[] for rpc in range(6)] for section in range(len(hv_data))]
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        try:
            event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        except:
            print(f'Error at event chunk {processedEvents}')
            fReader = rawFileReader.fileReader(file_path[1]) # load in the classs object
            continue
        #Zone of Reconstruction
        if event_chunk:
            # We need to update the event like TAnalyser as well
            event_time = max([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
            if event_time <  datetime.strptime(date+" "+hv_data["start"][0], '%Y-%m-%d %H:%M:%S'): 
                continue
            elif datetime.strptime(date+" "+hv_data["start"][measurement], '%Y-%m-%d %H:%M:%S') < event_time < datetime.strptime(date+" "+hv_data["end"][measurement], '%Y-%m-%d %H:%M:%S'):
                reconstructor.update_event(event_chunk, processedEvents)
                reconstructor.populate_hits()
                clusters = reconstructor.make_cluster()
                for evt in clusters:
                    for rpc in range(6):
                        if len(evt[2][rpc][0]):
                            historgrams[measurement][rpc].append(len(evt[2][rpc][0]))
                        if len(evt[2][rpc][1]):
                            historgrams[measurement][rpc].append(len(evt[2][rpc][1]))
                    
            elif event_time > datetime.strptime(date+" "+hv_data.iloc[-1]["end"], '%Y-%m-%d %H:%M:%S'):
                break 
            elif event_time > datetime.strptime(date+" "+hv_data["end"][measurement], '%Y-%m-%d %H:%M:%S'):
                measurement += 1
                total_clusters = 0
                num_clusters = 0
                print(f"Measurement {measurement} started")
                print(event_time)
                reconstructor.update_event(event_chunk, processedEvents)
                reconstructor.populate_hits()
                clusters = reconstructor.make_cluster()
                for evt in clusters:
                    for rpc in range(6):
                        if len(evt[2][rpc][0]):
                            historgrams[measurement][rpc].append(len(evt[2][rpc][0]))
                        if len(evt[2][rpc][1]):
                            historgrams[measurement][rpc].append(len(evt[2][rpc][1]))

        pbar.update(1)
print(event_time)

C:\Users\jony\Programming\Python\Anubis\anubis\data\proAnubis_240815_1759.raw
2024-08-15 16:59:49.312501
2024-08-15 17:06:33.650217
2024-08-15 17:13:38.548513
2024-08-15 17:18:30.477887


Processing Events:   0%|          | 5/10000000000 [00:01<597888:58:36,  4.65Events/s] 

Measurement 1 started
2024-08-15 17:21:39.643110
Measurement 2 started
2024-08-15 17:28:07.812363
Measurement 3 started
2024-08-15 17:29:00.939168
Measurement 4 started
2024-08-15 17:29:52.527109
Measurement 5 started
2024-08-15 17:29:56.885155


Processing Events:   0%|          | 41/10000000000 [00:02<64202:46:38, 43.27Events/s] 

Measurement 6 started
2024-08-15 17:31:18.153406


...

Measurement 23 started
2024-08-15 18:07:14.035057


Processing Events:   0%|          | 23185/10000000000 [12:18<87252:31:11, 31.84Events/s] 

Measurement 24 started
2024-08-15 18:08:55.080421


Processing Events:   0%|          | 24471/10000000000 [13:09<89593:51:07, 31.00Events/s] 

2024-08-15 18:10:34.027245


In [4]:
print(event_time)

2024-08-15 18:10:34.027245


In [6]:
import csv

with open("historgrams1.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(historgrams)

In [15]:
import matplotlib.pyplot as plt
import numpy as np
historgrams = np.array(historgrams, dtype=object)
#print(historgrams[0][:20])
for rpc in range(3):
    avg = [np.mean(measurement[rpc]) for measurement in historgrams]
    #print(avg)
    errors = np.array([np.std(measurement[rpc]) for measurement in historgrams])/np.array([np.sqrt(len(row[rpc])) for row in historgrams])
    plt.plot(hv_data["voltage"], avg, label=f"RPC {rpc}")
    plt.errorbar(hv_data["voltage"], avg, yerr=errors, fmt="x", color="r")

plt.legend()
plt.xlabel("Voltage")
plt.ylabel("Average Cluster Size")
plt.title(f"Cluster size - RPC 0 tested")
plt.show()


KeyboardInterrupt: 

In [17]:
# plot historgam
import matplotlib.pyplot as plt
import numpy as np
lower = 9
middle = 15
upper = len(hv_data)-1
counts, bins = np.histogram(historgrams[lower][0], bins=range(min(historgrams[lower][0])-1, max(historgrams[lower][0]) + 2), density=True)
plt.step(bins[:-1], counts, where='mid', label=f"Voltage {hv_data['voltage'][lower]}")

counts, bins = np.histogram(historgrams[middle][0], bins=range(min(historgrams[middle][0])-1, max(historgrams[middle][0]) + 2), density=True)
plt.step(bins[:-1], counts, where='mid', label=f"Voltage {hv_data['voltage'][middle]}")

counts, bins = np.histogram(historgrams[middle][1], bins=range(min(historgrams[middle][0])-1, max(historgrams[middle][0]) + 2), density=True)
plt.step(bins[:-1], counts, where='mid', label=f"Mid Triplet")

counts, bins = np.histogram(historgrams[-1][0], bins=range(min(historgrams[-1][0])-1, max(historgrams[-1][0]) + 2), density=True)
plt.step(bins[:-1], counts, where='mid', label=f"Voltage {hv_data['voltage'][len(hv_data)-1]}")

# Adding titles and labels
plt.title('Cluster size distribution - low triplet')
plt.xlabel('Value')
plt.yscale('log')
plt.ylabel('Frequency')
plt.legend()
# Show the plot
plt.show()